In [2]:



!pip install -q opendatasets

import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database/download?datasetVersionNumber=1")

Skipping, found downloaded files in "./anime-recommendations-database" (use force=True to force download)


In [3]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn


In [4]:
anime_data = pd.read_csv("/content/anime-recommendations-database/anime.csv")
anime_data['rating'].fillna(-1)
anime_data.head(10)

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
5,32935,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,"Comedy, Drama, School, Shounen, Sports",TV,10,9.15,93351
6,11061,Hunter x Hunter (2011),"Action, Adventure, Shounen, Super Power",TV,148,9.13,425855
7,820,Ginga Eiyuu Densetsu,"Drama, Military, Sci-Fi, Space",OVA,110,9.11,80679
8,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,"Action, Comedy, Historical, Parody, Samurai, S...",Movie,1,9.10,72534
9,15417,Gintama&#039;: Enchousen,"Action, Comedy, Historical, Parody, Samurai, S...",TV,13,9.11,81109


In [5]:
user_data = pd.read_csv("/content/anime-recommendations-database/rating.csv")
user_data.fillna(-1, inplace=True)
user_data.head(10)

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
5,1,355,-1
6,1,356,-1
7,1,442,-1
8,1,487,-1
9,1,846,-1


In [15]:
!pip install surprise
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

reader = Reader(rating_scale=(1, 10))


  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162999 sha256=135b425d2002ed6e6c757956f50a018143cb349acc922080fdd45abc3a317023
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [17]:
merge_data = pd.merge(anime_data, user_data, on='anime_id')

data = Dataset.load_from_df(merge_data[['user_id', 'anime_id', 'rating_y']], reader)

In [18]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [19]:
model = SVD()
model.fit(trainset)

predictions = model.test(testset)

accuracy = rmse(predictions)
print("RMSE:", accuracy)

RMSE: 2.2108
RMSE: 2.210824285391515
